<a href="https://colab.research.google.com/github/tolya732/Data-analyses-projects/blob/main/Credit_Screening_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загружаем spark

In [ ]:
!apt-get update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
!tar -xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

**ЗАДАНИЕ**

Cоздайте программу для определения кредитной платежеспособности (кредитного скрининга). Пусть имеются данные о клиентах, обратившихся за кредитом. Здесь объектами являются клиенты, а признаками могут являться их пол, уровень дохода, образование, информация о том, имеется или отсутствует у них положительная кредитная история и т.д.

В качестве выделенного признака (ответа) выступает информация о том, вернул ли клиент кредит в банк или нет. По этим данным требуется научиться предсказывать, вернет ли кредит новый клиент, обратившийся в банк.

Таким образом, речь идет о задаче классификации: требуется определить, какому классу: положительному (кредит будет возвращен) или отрицательному (кредит не будет возвращен) – принадлежит клиент.

Для построения модели используем файл данных по кредитным рейтингам crx.csv с сайта https://www.kaggle.com/code/kerneler/starter-uci-credit-approval-612a845f-d/data

Для начала импортируем нужные библиотеки и сами данные.

In [ ]:
import numpy as np
import pandas as pd
import sys

In [ ]:
df=pd.read_csv("crx.csv")

Проверка данных из csv

In [ ]:
df.head(10)

,b,30.83,0,u,g,w,v,1.25,t,t.1,1,f,g.1,202,0.1,+
0,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,43,560,+
1,a,24.5,0.500,u,g,q,h,1.500,t,f,0,f,g,280,824,+
2,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,100,3,+
3,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,120,0,+
4,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,360,0,+
5,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,164,31285,+
6,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,80,1349,+
7,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,180,314,+
8,b,42.5,4.915,y,p,w,v,3.165,t,f,0,t,g,52,1442,+
9,b,22.08,0.830,u,g,c,h,2.165,f,f,0,t,g,128,0,+


Проверим, сколько записей в датасете и есть ли у нас пропуски.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b       689 non-null    object 
 1   30.83   689 non-null    object 
 2   0       689 non-null    float64
 3   u       689 non-null    object 
 4   g       689 non-null    object 
 5   w       689 non-null    object 
 6   v       689 non-null    object 
 7   1.25    689 non-null    float64
 8   t       689 non-null    object 
 9   t.1     689 non-null    object 
 10  1       689 non-null    int64  
 11  f       689 non-null    object 
 12  g.1     689 non-null    object 
 13  202     689 non-null    object 
 14  0.1     689 non-null    int64  
 15  +       689 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.2+ KB


Результат показывает, что в наборе 689 записей, и нет ни одного NA.

In [ ]:
df.value_counts().to_frame()

0
b 30.83 0      u g w  v 1.25  t t.1 1 f g.1 202 0.1  +   
? 20.08 0.125  u g q  v 1.000 f t   1 f g   240 768  +  1
b 30.17 6.500  u g cc v 3.125 t t   8 f g   330 1200 +  1
  29.67 1.415  u g w  h 0.750 t t   1 f g   240 100  +  1
  29.83 1.250  y p k  v 0.250 f f   0 f g   224 0    -  1
        2.040  y p x  h 0.040 f f   0 f g   128 1    -  1
...                                                    ..
  16.5  0.125  u g c  v 0.165 f f   0 f g   132 0    -  1
  16.92 0.335  y p k  v 0.290 f f   0 f s   200 0    -  1
  17.08 0.085  y p c  v 0.040 f f   0 f g   140 722  -  1
        0.250  u g q  v 0.335 f t   4 f g   160 8    -  1
  ?     10.500 u g x  v 6.500 t f   0 f g   0   0    +  1

[689 rows x 1 columns]

Исследуем данные по столбцам, выясняем какие столбцы оставить для модели.

In [ ]:
df['g.1'].value_counts()

g    624
s     57
p      8
Name: g.1, dtype: int64

После исследования оставляем следующие 8 столбцов факторов для принятия решения и 9-й столбец (t.1) как решение о выдаче кредита: b (семейный статус), v (количество детей), + (итоговое решение выдать кредит или нет), 0 (месячный доход как отношение к прожит минимуму цифра от 0 до 28), g (опыт успешного взятия и погашения кредита), t.1 (пол), g.1 (есть ли страховка жизни), worldwide (возрастной промежуток - по 5 лет, начиная от 20 и старше 85)

In [ ]:
df1=df[['v','b','+','0','g','t','t.1','g.1','w']]

Очищаем столбец b (брак) от данных без ответа и столбец v (кол-во детей) от данных без ответа, и столбец опыт взятия и отдачи кредита (g)

In [ ]:
df2=df1.loc[(df1['b']!='?')&(df1['v']!='?')&(df1['g']!='?')&(df1['g']!='gg')&(df1['w']!='?')&(df1['g.1']!='p')]
df2.head()

,v,b,+,0,g,t,t.1,g.1,w
0,h,a,+,4.460,g,t,t,g,q
1,h,a,+,0.500,g,t,f,g,q
2,v,b,+,1.540,g,t,t,g,w
3,v,b,+,5.625,g,t,f,s,w
4,v,b,+,4.000,g,t,f,g,m


Создаем spark DataFrame из Pandas DataFrame

In [ ]:
df3 = spark.createDataFrame(df2)
df3.describe().show()

/content/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-------+----+----+----+-----------------+----+----+----+----+----+
|summary|   v|   b|   +|                0|   g|   t| t.1| g.1|   w|
+-------+----+----+----+-----------------+----+----+----+----+----+
|  count| 667| 667| 667|              667| 667| 667| 667| 667| 667|
|   mean|null|null|null| 4.82330584707646|null|null|null|null|null|
| stddev|null|null|null|4.963728352011556|null|null|null|null|null|
|    min|  bb|   a|   +|              0.0|   g|   f|   f|   g|  aa|
|    max|   z|   b|   -|             28.0|   p|   t|   t|   s|   x|
+-------+----+----+----+-----------------+----+----+----+----+----+



Переименовываем столбцы

In [ ]:
df3=df3.withColumnRenamed('v','chld').withColumnRenamed('b','mrg').withColumnRenamed('+','res'). \
        withColumnRenamed('0','inc').withColumnRenamed('g','crex').withColumnRenamed('t.1','sex'). \
        withColumnRenamed('t','ed').withColumnRenamed('g.1','ins').withColumnRenamed('w','age')

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

Создаём индексаторы

In [ ]:
indexer_chld = StringIndexer(inputCol='chld', outputCol='chldInd')
indexer_mrg = StringIndexer(inputCol='mrg', outputCol='mrgInd')
indexer_res = StringIndexer(inputCol='res', outputCol='resInd')
indexer_crex = StringIndexer(inputCol='crex', outputCol='crexInd')
indexer_ed = StringIndexer(inputCol='ed', outputCol='edInd')
indexer_sex = StringIndexer(inputCol='sex', outputCol='sexInd')
indexer_ins = StringIndexer(inputCol='ins', outputCol='insInd')
indexer_age = StringIndexer(inputCol='age', outputCol='ageInd')

Создаём вектор преобразователь

In [ ]:
vectorAssembler = VectorAssembler(inputCols=["chldInd", "mrgInd", "inc", "crexInd", "edInd", "sexInd", "insInd", "ageInd"],
                                  outputCol="features")

Создаём конвеер

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer_chld, indexer_mrg, indexer_res, indexer_crex, indexer_ed, indexer_sex, indexer_ins, indexer_age, vectorAssembler])

df_transformed = pipeline.fit(df3).transform(df3)
df_transformed.show(3)

+----+---+---+----+----+---+---+---+---+-------+------+------+-------+-----+------+------+------+--------------------+
|chld|mrg|res| inc|crex| ed|sex|ins|age|chldInd|mrgInd|resInd|crexInd|edInd|sexInd|insInd|ageInd|            features|
+----+---+---+----+----+---+---+---+---+-------+------+------+-------+-----+------+------+------+--------------------+
|   h|  a|  +|4.46|   g|  t|  t|  g|  q|    1.0|   1.0|   1.0|    0.0|  0.0|   1.0|   0.0|   1.0|[1.0,1.0,4.46,0.0...|
|   h|  a|  +| 0.5|   g|  t|  f|  g|  q|    1.0|   1.0|   1.0|    0.0|  0.0|   0.0|   0.0|   1.0|(8,[0,1,2,7],[1.0...|
|   v|  b|  +|1.54|   g|  t|  t|  g|  w|    0.0|   0.0|   1.0|    0.0|  0.0|   1.0|   0.0|   2.0|(8,[2,5,7],[1.54,...|
+----+---+---+----+----+---+---+---+---+-------+------+------+-------+-----+------+------+------+--------------------+
only showing top 3 rows



Создаём финальную таблицу

In [ ]:
df_final = df_transformed.select(["features", "resInd"])
df_final.show(3)

+--------------------+------+
|            features|resInd|
+--------------------+------+
|[1.0,1.0,4.46,0.0...|   1.0|
|(8,[0,1,2,7],[1.0...|   1.0|
|(8,[2,5,7],[1.54,...|   1.0|
+--------------------+------+
only showing top 3 rows



Разбиваем данные на тренировочную и тестовыую выборки

In [ ]:
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=12345)

Создаём модель

In [ ]:
from pyspark.ml.classification import LogisticRegression

regressor = LogisticRegression(featuresCol="features", labelCol="resInd")

Обучаем модель

In [ ]:
regressor = regressor.fit(train_data)

Получаем результаты для тренировочной и тестовой выборки

In [ ]:
train_res = regressor.transform(train_data)
test_res = regressor.transform(test_data)

In [ ]:
train_res.show(3)

+--------------------+------+--------------------+--------------------+----------+
|            features|resInd|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|       (8,[0],[2.0])|   1.0|[-0.1915052800707...|[0.45226946445146...|       1.0|
|(8,[0,1,2],[1.0,1...|   1.0|[-0.5889606607033...|[0.35687336251942...|       1.0|
|(8,[0,1,2,5],[1.0...|   1.0|[-2.1124869877899...|[0.10788906310943...|       1.0|
+--------------------+------+--------------------+--------------------+----------+
only showing top 3 rows



In [ ]:
test_res.show(3)

+--------------------+------+--------------------+--------------------+----------+
|            features|resInd|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(8,[0,1,2,7],[1.0...|   1.0|[-0.9868887961861...|[0.27152703646680...|       1.0|
|(8,[0,2,4],[1.0,0...|   1.0|[3.83498470851394...|[0.97885509641456...|       0.0|
|(8,[0,2,4],[1.0,1...|   0.0|[3.81622087707238...|[0.97846321684578...|       0.0|
+--------------------+------+--------------------+--------------------+----------+
only showing top 3 rows



Оцениваем модель

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

ev = BinaryClassificationEvaluator(labelCol='resInd')

ev.evaluate(train_res)

0.9309053497942374

In [ ]:
ev.evaluate(test_res)

0.8798941798941793

**Вывод:** Логистическая регрессия даёт 88% вероятности результата успеха на тестовой базе.

**ПРОГРАММА**, которая по заданным параметрам дает прогноз вернет клиент кредит или нет. В итоге печатается результат.

In [ ]:
from numpy.lib.function_base import vectorize
a1=float(input('Введите действительно число от 0 до 28 кратное делению вашего месячного дохода на прожиточный минимум: '))
if (a1<0)or(a1>28):
  print('введены неправильные данные. Вводимое число должно быть от 0 до 28')
  sys.exit()
a3=int(input('Введите 1, если Вы состоите в браке, 0 если нет: '))
if (a3!=0)&(a3!=1):
  print('введены неправильные данные. Вводимое число должно быть или 0 или 1')
  sys.exit()
a5=int(input('Введите 1, если Вы брали и успешно отдали кредит, 0 если нет: '))
if (a5!=0)&(a5!=1):
  print('введены неправильные данные. Вводимое число должно быть или 0 или 1')
  sys.exit()
a6=int(input('Введите 1, если у Вас высшее образование, 0 если нет: '))
if (a6!=0)&(a6!=1):
  print('введены неправильные данные. Вводимое число должно быть или 0 или 1')
  sys.exit()
a2=int(input('Введите кол-во ваших детей - число от 0 до 8: '))
if (a2<0)or(a2>8):
  print('введены неправильные данные. Вводимое число должно быть от 0 до 8')
  sys.exit()
a4=int(input('Введите 1, если Вы мужчина, 0 если нет: '))
if (a4!=0)&(a4!=1):
  print('введены неправильные данные. Вводимое число должно быть или 0 или 1')
  sys.exit()
a7=int(input('Введите 0, если у Ваc есть мед. страхование, 1 если нет: '))
if (a7!=0)&(a7!=1):
  print('введены неправильные данные. Вводимое число должно быть или 0 или 1')
  sys.exit()
a8=int(input('Введите целое число от 0 до 13 в зависимости от возраста 0 если Вам (20-25 лет),13 (если Вы старше 85 лет): '))
if (a8<0)or(a8>13):
  print('введены неправильные данные. Вводимое число должно быть или 0 или 1')
  sys.exit()

test0 = pd.DataFrame({'inc':[a1],'chldInd':[a2],'mrgInd':[a3],'sexInd':[a4],'crexInd':[a5],'edInd':[a6],'insInd':[a7],'ageInd':[a8]})
test1 = spark.createDataFrame(test0)

df_test = vectorAssembler.transform(test1)
df_test_final = df_test.select(["features"])

test_res = regressor.transform(df_test_final)

test_res.show()

Введите действительно число от 0 до 28 кратное делению вашего месячного дохода на прожиточный минимум: 10
Введите 1, если Вы состоите в браке, 0 если нет: 0
Введите 1, если Вы брали и успешно отдали кредит, 0 если нет: 0
Введите 1, если у Вас высшее образование, 0 если нет: 0
Введите кол-во ваших детей - число от 0 до 8: 0
Введите 1, если Вы мужчина, 0 если нет: 0
Введите 0, если у Ваc есть мед. страхование, 1 если нет: 0
Введите целое число от 0 до 13 в зависимости от возраста 0 если Вам (20-25 лет),13 (если Вы старше 85 лет): 5


/content/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(8,[2,7],[10.0,5.0])|[-0.9241282211077...|[0.28411748805032...|       1.0|
+--------------------+--------------------+--------------------+----------+



In [ ]:
if test_res.first()['prediction']:
  print('Кредит одобрен')
else:
  print('Кредит отклонен')

Кредит одобрен


Попробуем, а будет ли улучшение результата при применении не логистической, а линейной регрессии :

In [ ]:
from pyspark.ml.regression import LinearRegression

lin_reg = LinearRegression(featuresCol='features',labelCol='resInd')
lrMod=lin_reg.fit(train_data)

train_res1=lrMod.transform(train_data)
test_res1=lrMod.transform(test_data)

from pyspark.ml.evaluation import RegressionEvaluator

ev1 = RegressionEvaluator(predictionCol='prediction',
      labelCol='resInd')

ev1.evaluate(train_res1)

0.31578830609967595

In [ ]:
ev1.evaluate(test_res1)

0.36042673852791446

Линейная регрессия дает очень слабый результат. Ее использовать нельзя.